# Automated ML


## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


## Initialize Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-144884
aml-quickstarts-144884
southcentralus
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


## Create an Azure ML experiment

In [3]:
# choose a name for experiment
experiment_name = 'loan-prediction'
project_folder = './loan-prediction-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
loan-prediction,quick-starts-ws-144884,Link to Azure Machine Learning studio,Link to Documentation


## Create a compute cluster


In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.


## Dataset

### Overview
In this project, we use a [loan prediction problem dataset](https://www.kaggle.com/altruistdelhite04/loan-prediction-problem-dataset) from Kaggle.
The dataset contains 11 features and the target column **Loan_Status**.

In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "raw-loan-prediction-dataset"
description_text = "Loan prediction dataset before cleaning"

if key in ws.datasets.keys(): 
        found = True
        raw_dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/fnakashima/nd00333-capstone/master/starter_file/dataset/train_u6lujuX_CVtuZ9i.csv'
        raw_dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        raw_dataset = raw_dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = raw_dataset.to_pandas_dataframe()
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,False,0,Graduate,False,5849,0.0,NaN,360.0,1.0,Urban,True
1,LP001003,Male,True,1,Graduate,False,4583,1508.0,128.0,360.0,1.0,Rural,False
2,LP001005,Male,True,0,Graduate,True,3000,0.0,66.0,360.0,1.0,Urban,True
3,LP001006,Male,True,0,Not Graduate,False,2583,2358.0,120.0,360.0,1.0,Urban,True
4,LP001008,Male,False,0,Graduate,False,6000,0.0,141.0,360.0,1.0,Urban,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,False,0,Graduate,False,2900,0.0,71.0,360.0,1.0,Rural,True
610,LP002979,Male,True,3+,Graduate,False,4106,0.0,40.0,180.0,1.0,Rural,True
611,LP002983,Male,True,1,Graduate,False,8072,240.0,253.0,360.0,1.0,Urban,True
612,LP002984,Male,True,2,Graduate,False,7583,0.0,187.0,360.0,1.0,Urban,True


## Clean dataset

In [7]:
def clean_data(data):
    # Dict for cleaning data
    dependents = {"0":0, "1":1, "2":2, "3+":3}
    property_areas = {"Urban":1, "Semiurban":2, "Rural":3}

    # Clean and one hot encode data
    x_df = data.dropna()
    x_df.drop("Loan_ID", axis=1, inplace=True)

    # Filtering "True", "Yes", "Y" won't work as it will be recoginised as a boolean value automatically by dataset framework
    x_df.loc[:,('Gender')] = x_df.Gender.apply(lambda s: 1 if s == "Male" else 2)
    x_df.loc[:,('Married')] = x_df.Married.apply(lambda s: 1 if s else 0)
    x_df.loc[:,('Dependents')] = x_df.Dependents.map(dependents)
    x_df.loc[:,('Education')] = x_df.Education.apply(lambda s: 1 if s == "Graduate" else 0)
    x_df.loc[:,('Self_Employed')] = x_df.Self_Employed.apply(lambda s: 1 if s else 0)
    x_df.loc[:,('Property_Area')] = x_df.Property_Area.map(property_areas)

    y_df = x_df.pop("Loan_Status").apply(lambda s: 1 if s else 0)
    return x_df, y_df

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd

df
x, y = clean_data(df)
x

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
1,1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,3
2,1,1,0,1,1,3000,0.0,66.0,360.0,1.0,1
3,1,1,0,0,0,2583,2358.0,120.0,360.0,1.0,1
4,1,0,0,1,0,6000,0.0,141.0,360.0,1.0,1
5,1,1,2,1,1,5417,4196.0,267.0,360.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
609,2,0,0,1,0,2900,0.0,71.0,360.0,1.0,3
610,1,1,3,1,0,4106,0.0,40.0,180.0,1.0,3
611,1,1,1,1,0,8072,240.0,253.0,360.0,1.0,1
612,1,1,2,1,0,7583,0.0,187.0,360.0,1.0,1


In [8]:
y

1      0
2      1
3      1
4      1
5      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 478, dtype: int64

## Split train and test data

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd


# Split data into train and test sets.
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# Default test_size: 0.25
x_train, x_test, y_train, y_test = train_test_split(x, y)

# concatenate train data x and y and test
# https://pandas.pydata.org/docs/user_guide/merging.html
train_data = pd.concat([x_train, y_train], axis=1)
pd.DataFrame(train_data)

test_data = pd.concat([x_test, y_test], axis=1)
pd.DataFrame(test_data)

train_data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
536,1,1,0,1,0,6133,3906.0,324.0,360.0,1.0,1,1
480,2,0,0,0,0,4350,0.0,154.0,360.0,1.0,3,1
209,1,0,0,1,0,3418,0.0,127.0,360.0,1.0,2,0
106,1,1,2,1,0,11417,1126.0,225.0,360.0,1.0,1,1
178,1,1,1,1,0,3750,0.0,116.0,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
164,1,1,0,1,0,9323,0.0,75.0,180.0,1.0,1,1
9,1,1,1,1,0,12841,10968.0,349.0,360.0,1.0,2,0
287,1,1,0,1,0,3704,2000.0,120.0,360.0,1.0,3,1
157,1,1,1,1,0,9538,0.0,187.0,360.0,1.0,1,1


### Register training and test data for use in AutoML

In [10]:
# To specify train and test dataset in AutoMLConfig, register the datasets in the cloud.
from azureml.data.dataset_factory import TabularDatasetFactory

ds_name = "loan-prediction-dataset"
ds_description = "Loan prediction dataset after cleaning"

#if key in ws.datasets.keys(): 
#    found = True
#    train_ds = ws.datasets[key] 
#    print('Found existing train dataset, use it.')
#
#if not found:
#    datastore = ws.get_default_datastore()
#    train_ds = TabularDatasetFactory.register_pandas_dataframe(train_data, datastore, ds_name, description=ds_description)
datastore = ws.get_default_datastore()
train_ds = TabularDatasetFactory.register_pandas_dataframe(train_data, datastore, ds_name, description=ds_description)

train_df = train_ds.to_pandas_dataframe()
#train_df

test_ds_name = "loan-prediction-dataset-test"
test_ds_description = "Loan prediction dataset after cleaning for testing"

test_ds = TabularDatasetFactory.register_pandas_dataframe(test_data, datastore, test_ds_name, description=test_ds_description)

test_df = test_ds.to_pandas_dataframe()
#test_df

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a774b1ae-a004-49c9-ba7d-724cff146f87/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/bab1c894-dd48-494f-8ad0-4a15afb31596/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

Since the target problem is to predict customer's loan status (binary prediction: 0 or 1), the task type is **classification** and the target label column is **Loan_Status**.

We use **accuracy** as a primary metric and set **30** as **experiment_timeout_minutes** to limit experiment running duration.

We set **10** as **max_concurrent_iterations** to run iterations in the experiment in parallel.
**featurization** is set to **auto** to enable featurization step to be done automatically.

Reference: [AutoMLConfig Class](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py)


In [11]:
automl_settings = {
    "primary_metric" : 'accuracy',
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "featurization" : 'auto'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_ds,
                             validation_data=test_ds,
                             label_column_name="Loan_Status",   
                             path = project_folder,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
loan-prediction,AutoML_5bf3294d-82f2-4210-8187-31b69e2c2363,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [14]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
loan-prediction,AutoML_5bf3294d-82f2-4210-8187-31b69e2c2363,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_5bf3294d-82f2-4210-8187-31b69e2c2363',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-15T18:01:03.39432Z',
 'endTimeUtc': '2021-05-15T18:38:18.284523Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"loan-prediction","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-144884","workspace_name":"quick-starts-ws-144884","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","d

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
# Retrieve best model from Pipeline Run
best_automl_run, fitted_automl_model = remote_run.get_output()
print(best_automl_run)
best_automl_run_metrics = best_automl_run.get_metrics()
print('Accuracy: ', best_automl_run_metrics['accuracy'])
print('Other metrics:')
# Display all metrics of the best run
for metric_name in best_automl_run_metrics:
    if metric_name == 'accuracy':
        continue

    metric = best_automl_run_metrics[metric_name]
    print(metric_name, ": ", metric)

Run(Experiment: loan-prediction,
Id: AutoML_5bf3294d-82f2-4210-8187-31b69e2c2363_282,
Type: azureml.scriptrun,
Status: Completed)
Accuracy:  0.875
Other metrics:
average_precision_score_weighted :  0.8118777407471434
log_loss :  0.47528584009252434
f1_score_macro :  0.7948717948717949
recall_score_micro :  0.875
balanced_accuracy :  0.7616487455197133
f1_score_micro :  0.875
precision_score_macro :  0.857843137254902
precision_score_weighted :  0.8713235294117647
weighted_accuracy :  0.9357005758157391
average_precision_score_micro :  0.8329387566833487
AUC_macro :  0.711668657905217
precision_score_micro :  0.875
f1_score_weighted :  0.8653846153846153
recall_score_weighted :  0.875
matthews_correlation :  0.6119778033738925
average_precision_score_macro :  0.73164215807519
norm_macro_recall :  0.5232974910394266
AUC_micro :  0.8471527777777776
AUC_weighted :  0.7116686579052172
recall_score_macro :  0.7616487455197133
accuracy_table :  aml://artifactId/ExperimentRun/dcid.AutoML_5bf32

In [32]:
fitted_automl_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('270',
                                             Pipeline(memory=None,
                                                      steps=[('standardscalerwrapper',
                                                              <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f62160de518>),
                                                             ('gradientboostingclassifier',
                                                              GradientBo

### Regiser and save the best model

In [16]:
# Retrieve and save your best automl model.
best_automl_run, fitted_automl_model = remote_run.get_output()
best_automl_run_metrics = best_automl_run.get_metrics()

automl_model = remote_run.register_model(model_name = 'loan-prediction-automl-model',
                                  tags={'Method':'AutoML'},
                                  description='AutoML Model trained on loan prediction data to predict a loan status of customers')

In [18]:
# Save the model in the local project folder
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', project_folder + '/score.py')
best_automl_run.download_file('outputs/model.pkl', project_folder + '/outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
import sklearn


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [21]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model

service_name = 'loan-prediction-automl-service'

inference_config = InferenceConfig(entry_script=project_folder + '/score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=1,
                                                enable_app_insights=True,
                                                description="Loan status prediction service")

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[automl_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-15 18:53:16+00:00 Creating Container Registry if not exists.
2021-05-15 18:53:17+00:00 Registering the environment.
2021-05-15 18:53:17+00:00 Use the existing image.
2021-05-15 18:53:17+00:00 Generating deployment configuration.
2021-05-15 18:53:18+00:00 Submitting deployment to compute.
2021-05-15 18:53:21+00:00 Checking the status of deployment loan-prediction-automl-service..
2021-05-15 18:54:40+00:00 Checking the status of inference endpoint loan-prediction-automl-service.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 993ec790-49aa-4648-b95d-b29875dcb51e
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: loan-prediction-automl-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 3c9f512a451e46feb66962f4f34f6621.azurecr.io/azureml/azureml_73804c8efe1927e444374f2dcb6e4227 locally. Ple

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 993ec790-49aa-4648-b95d-b29875dcb51e
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: loan-prediction-automl-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 3c9f512a451e46feb66962f4f34f6621.azurecr.io/azureml/azureml_73804c8efe1927e444374f2dcb6e4227 l

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service